# TF-IDF

We'll rely on term frequency times inverted document frequency to measure meaningful similarity between documents. Let's start by generating a matrix for the separate constituent parts of _Stjórn_.

In [1]:
import os,glob,json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def normalize(target):
    # This dict further standardizes the text beyond the rule set of stjorn-extract.ipynb:
    matrix = {
        'j': 'i',
        'v': 'u',
        'ð': 'þ',
        'á': 'a',
        'ǽ': 'æ',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'ý': 'y',
        'ǿ': 'ø',
        'k': 'c',
        '[': '',
        ']': ''
        }
    for k,v in matrix.items():
        target = target.replace(k, v)
    return target

titles = ['prologue', 'introduction', 'gn', 'ex', 'lv', 'nm', 'dt', 'ios', 'idc', 'rt', '1sm', '2sm', '3rg', '4rg']
tokens = []
for title in titles:
    with open(f"nlp/{title}.txt") as raw:
        document = raw.read().replace('\n', ' ')
        tokens.extend(document.split())

work_indices = {
    'stjorn1': (650,124417),
    'stjorn2': (124417,147678),
    'stjorn3': (147678,156943,160719),
    'stjorn4': (156943,160719)
}

stjorn = dict()
for _work, _range in work_indices.items():
    if len(_range) == 2:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]]))
    else:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]] + tokens[_range[2]:]))

menota = dict()
for text in glob.glob('../menota/dipl/*txt'):
    ref = os.path.basename(text).replace('.txt', '')
    with open(text) as doc:
        # NB for present purposes I'm subjecting Menota to the same normalization standard as Stjórn:
        menota[ref] = normalize(doc.read().replace('\n', ''))

In [3]:
vectorizer = TfidfVectorizer(min_df=1)
model = vectorizer.fit_transform(stjorn.values())
df = pd.DataFrame(cosine_similarity(model), stjorn.keys(), stjorn.keys())
df

,stjorn1,stjorn2,stjorn3,stjorn4
stjorn1,1.000000,0.811733,0.903770,0.842752
stjorn2,0.811733,1.000000,0.797618,0.833766
stjorn3,0.903770,0.797618,1.000000,0.835339
stjorn4,0.842752,0.833766,0.835339,1.000000


After eliminating vowel length and the þ/ð distinction, these are now all pretty similar to one another, with the biggest difference between _Stjórn II_ and _III_.

Now let's first add _Konungs skuggsjá_ from Menota, as well as Unger's own edition of the _Norwegian Homily Book_. Fingers crossed that we have got the normalization standard of the former to approach Unger's methods reasonably well.

In [4]:
nhb_titles = ['alcuin', 'hom', 'olafr', 'visio', 'paternoster', 'anhang1'] # this is the sequence matched in Menota
nhb = ''
for title in nhb_titles:
    filepath = f'../nhb/nlp/{title}.txt'
    with open(filepath) as doc:
        nhb = nhb + doc.read().replace('\n', '')
stjorn_plus = []
for v in stjorn.values():
    stjorn_plus.append(v)
stjorn_plus.extend([menota['nks235g_konungs_skuggsja'], nhb])
model = vectorizer.fit_transform(stjorn_plus)
df = pd.DataFrame(cosine_similarity(model), list(stjorn.keys()) + ['ks', 'nhb'], list(stjorn.keys()) + ['ks', 'nhb'])
df

,stjorn1,stjorn2,stjorn3,stjorn4,ks,nhb
stjorn1,1.000000,0.794566,0.883015,0.812126,0.705122,0.711161
stjorn2,0.794566,1.000000,0.773820,0.803064,0.690873,0.671303
stjorn3,0.883015,0.773820,1.000000,0.797769,0.739994,0.740641
stjorn4,0.812126,0.803064,0.797769,1.000000,0.656851,0.677968
ks,0.705122,0.690873,0.739994,0.656851,1.000000,0.664858
nhb,0.711161,0.671303,0.740641,0.677968,0.664858,1.000000


_Stjórn III_ and _Konungs skuggsjá_ share material cognate within the vernacular, so this connection standing out as the strongest between _Konungs skuggsjá_ and the constituent parts of _Stjórn_ comes as no surprise; if anything, the difference in match with the other parts is rather small. In fact, the _Norwegian Homily Book_ has a higher match with _Stjórn III_ than _Konungs skuggsjá_ does.

Next, let's model all of Menota along with Stjórn. Perhaps we'll leave Unger's _Homily Book_ in alongside the Menota edition, just for comparison's sake.

In [5]:
corpus = []
titles = []
for k,v in stjorn.items():
    titles.append(k)
    corpus.append(v)
titles.append('nhb')
corpus.append(nhb)
for k,v in menota.items():
    titles.append(k)
    corpus.append(v)
model = vectorizer.fit_transform(corpus)
df = pd.DataFrame(cosine_similarity(model), titles, titles)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df

,stjorn1,stjorn2,stjorn3,stjorn4,nhb,nraNorrFragm75_kross_saga,am132_egils_saga,am162btheta_njals_saga,nraNorrFragm64_barlaams_saga,nraNorrFragm81A_benedikts_regla,am1056IX_konungs_skuggsja_fragment,am78_kristinrettir,am63_heimskringla3,dg4-7_strengleikar,am132_droplaugasona_saga,am132_kormaks_saga,nraNorrFragm72x76_dialogar,nraNorrFragm53_haralds_saga_hardrada,am132_finnboga_saga,nraNorrFragm70_agotu_saga,nraNorrFragm62_karlamagnuss_saga,nraNorrFragm60A_stjorn,am132_fostbraedra_saga,lbsFragm82_olafs_saga_helga,nraNorrFragm58B_konungs_skuggsja,nraNorrFragm60C_stjorn,holmPerg30_landslog,am619_norwegian_homily_book,nhb_am619,nraNorrFragm57_jons_saga_helga,nraNorrFragm69_nikulass_saga,am56_landslog,wolfAug9-10_egils_saga,nraNorrFragm66_thomass_saga,holmPerg17_thomass_saga,am383I_thorlaks_saga,holmPerg4_thidreks_saga,am132_njals_saga,am36_heimskringla2,am544_voluspa,am162bkappa_njals_saga,am305_landslog,nraNorrFragm58C_konungs_skuggsja,am132_olkofra_thattr,konungs_skuggsja_am243ba,nraNorrFragm54_sverris_saga,nraNorrFragm55B_hakonar_saga,nraNorrFragm79_mariu_saga,gks2365_voluspa,am243balpha_konungs_skuggsja,nraNorrFragm51_fagrskinna,am132_viga-glums_saga,am279a_gragas,am677_gregory,am132_laxdoela_saga,am302_landslog,am178_thidreks_saga,nraNorrFragm81B_benedikts_regla,am132_bandamanna_saga,nraNorrFragm71_gregors_saga_pafa,gregory_am677,am655_laeknisbok,am519a_alexanders_saga,holmPerg34_landslog,am162balpha_njals_saga,am113_islendingabok,nraNorrFragm7_landslog,nraNorrFragm67_thomass_saga,nraNorrFragm56_thorgils_saga,nks235g_konungs_skuggsja,am132_hallfredar_saga,am35_heimskringla1,am242_codex_wormianus,nraNorrFragm78_mariu_saga,konungs_skuggsja_fragment_am1056xi,dg8II_olafs_saga,nraNorrFragm80_pals_saga,nraNorrFragm63_karlamagnuss_saga,nraNorrFragm77_dialogar,am28_codex_runicus,holmPerg34_boejarlog,dg8I_landslog,nraNorrFragm60B_stjorn,nraNorrFragm55A_hakonar_saga,skbA120_marys_complaint,nraNorrFragm59_rimbegla,nraNorrFragm65_floress_saga,nraNorrFragm52_olafs_saga_helga_hin_elzta,holmPerg6_barlaams_saga,nraNorrFragm68_brendanuss_saga,nraNorrFragm61_karlamagnuss_saga,nraNorrFragm58A_konungs_skuggsja
stjorn1,1.000000,0.675326,0.782961,0.649681,0.492555,0.257130,0.672272,0.571469,0.499756,0.222804,0.368045,0.420957,0.649526,0.728539,0.581550,0.476370,0.493932,0.391121,0.575159,0.233190,0.597929,0.356609,0.627396,0.573349,0.473806,0.484693,0.361864,0.631456,0.631456,0.605015,0.623539,0.423872,0.651883,0.601887,0.628798,0.423834,0.690073,0.675837,0.669838,0.310528,0.519057,0.412940,0.560838,0.506388,0.554455,0.668081,0.431882,0.444717,0.328786,0.554455,0.385541,0.655222,0.341152,0.512041,0.661015,0.401738,0.149348,0.356142,0.600724,0.367715,0.512041,0.455764,0.662860,0.553683,0.497843,0.366937,0.425181,0.669123,0.465248,0.545201,0.551810,0.662937,0.710050,0.637883,0.368045,0.617452,0.610302,0.641282,0.593190,0.227007,0.519909,0.469970,0.293146,0.511849,0.195335,0.432353,0.453836,0.605947,0.725827,0.488110,0.485022,0.582661
stjorn2,0.675326,1.000000,0.653911,0.628009,0.437318,0.252721,0.603981,0.493013,0.458417,0.198335,0.325308,0.412751,0.567410,0.627519,0.540143,0.438286,0.442036,0.342971,0.500490,0.214354,0.544420,0.280095,0.564592,0.507353,0.431038,0.390873,0.373688,0.574865,0.574865,0.529469,0.518215,0.411041,0.591957,0.488402,0.463665,0.341695,0.641929,0.622266,0.577102,0.285181,0.484935,0.399024,0.527441,0.482170,0.503279,0.615997,0.404818,0.391300,0.312032,0.503279,0.357702,0.596252,0.313254,0.478181,0.598276,0.389665,0.171301,0.325215,0.558047,0.347309,0.478181,0.425372,0.577992,0.563036,0.478668,0.309285,0.412338,0.579205,0.382742,0.506668,0.515591,0.572766,0.608345,0.559135,0.325308,0.556403,0.540826,0.603391,0.533631,0.197242,0.540580,0.464683,0.280110,0.433989,0.165237,0.412673,0.421713,0.557471,0.667936,0.444594,0.457095,0.539131
stjorn3,0.782961,0.653911,1.000000,0.617558,0.558765,0.283557,0.782888,0.650656,0.557949,0.266460,0.404185,0.492523,0.738019,0.817113,0.672016,0.549874,0.562407,0.476364,0.649489,0.273746,0.686655,0.3

After substantial normalization, the two editions of NHB come to 0.69 similarity, which is still concerning, even if it is 15 percentage points better than before collapsing vowel length and þ/ð. There's probably Menota elements to disable yet, along the lines of `<sic>` and `<note>`. Really we expect a match in the 0.9-range.